In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import DataLoader
from DataSet import ImSegDataSet
import time
import segmentation_models_pytorch as smp
import numpy as np
import utils.unet_utils as ut

In [ ]:
def initialize_weights(m):
  if isinstance(m, nn.Conv2d):
      nn.init.xavier_normal_(m.weight.data)
  elif isinstance(m, nn.ConvTranspose2d):
      nn.init.xavier_normal_(m.weight.data)

In [ ]:
#Losses 

from utils.unet_utils import train

#Use the cross entropy loss, since we have more than two classes per pixel
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def CriterionCombo(inputs,targets):

    #Get One Hot encoding of the true labels
    y_true = torch.nn.functional.one_hot(targets,24).permute(0,3,1,2).float()

    assert y_true.shape == inputs.shape
    
    BCELoss = smp.losses.SoftBCEWithLogitsLoss()
    TverskyLoss = smp.losses.TverskyLoss(mode = 'multilabel', log_loss = False)

    L1 = BCELoss(y_pred = inputs,y_true = y_true)
    L2 = TverskyLoss(y_pred = inputs,y_true = y_true)

    loss = 0.5 * L1 + 0.5 * L2

    return loss

def DiceLoss(inputs,targets):
    
    #Use Tversky loss with alpha = beta = 0.5 for dice loss
    #This has the advantage of working better with possible class imbalance

    #Get One Hot encoding of the true labels
    y_true = torch.nn.functional.one_hot(targets,24).permute(0,3,1,2).float()

    assert y_true.shape == inputs.shape
    TverskyLoss = smp.losses.TverskyLoss(mode = 'multilabel', log_loss = False)

    loss = TverskyLoss(y_pred = inputs,y_true = y_true)

    return loss

def FocalLoss(inputs,targets):

    #Use Focal loss

    #Get One Hot encoding of the true labels
    y_true = torch.nn.functional.one_hot(targets,24).permute(0,3,1,2).float()

    assert y_true.shape == inputs.shape
    FocalLoss = smp.losses.FocalLoss(mode = 'multilabel')

    loss = FocalLoss(y_pred = inputs.contiguous(),y_true = y_true.contiguous())

    return loss

In [ ]:
#Set the training parameters here
config = {
    "lr": 0.0003,
    "loss":"combo",
    "batchSize":10,
    "patchSize":256,
    "nEpochs":3,
    "bufferSize":5,
    "bufferUpdateFreq":1,
    "bufferPickSize":3,
    "batchSizeDataLoaderTrain":1,
    "useRotation":False,
    "useMirroring":False,
    "scalingFactor":10,
    "useOriginal":True,
    "classCountThreshold":3,
    "modelParameters":{"in_channels":3, "out_channels":24, "sizes":[64, 128, 256, 512]},
    "modelName":"uNet"
}

In [ ]:
loss_dict = {
    "dice": DiceLoss,
    "combo":CriterionCombo,
    "focal": FocalLoss
}

model_dict = {
    "uNet":ut.UNet
}

In [ ]:
#batch_sizes = [1, 10]
#patch_sizes = 
#with open(f"results/result04.txt","a") as file:
#    file.write("threshold,accuracy,f1_score\n")
#file.close()
thresholds = [1,2,3,4,5,7]

for idx, threshold in enumerate(thresholds):

    #Set the iterable parameters
    #config["patchSize"] = patch_size
    #config["batchSize"] = batch_size
    config["classCountThreshold"] = threshold
        
    try:
        crit = loss_dict[config["loss"]]
        model = model_dict[config["modelName"]](**config["modelParameters"]).to(device)
        model.apply(initialize_weights)
        optimizer = optim.Adam(model.parameters(), lr = config["lr"])

        #Data set and data loader for the training set
        DS_training = ImSegDataSet(PathToDataSet="./data/train_set/",mode = "train")
        DL_training = DataLoader(dataset=DS_training,batch_size=config["batchSizeDataLoaderTrain"],shuffle=True)

        #Data set and data loader for the validation set
        print("\n")
        DS_validation = ImSegDataSet(PathToDataSet="./data/validation_set/",mode = "validation")
        DL_validation = DataLoader(dataset=DS_validation,batch_size=5)

        tag = "test{}".format(time.strftime("%d-%H%M%S", time.localtime(time.time())))

        result = train(
            model = model, 
            dataloader_training = DL_training, 
            dataLoader_validation = DL_validation,
            optimizer = optimizer, 
            criterion = crit, 
            device = device, 
            buffer_size = config["bufferSize"], 
            buffer_update_freq = config["bufferUpdateFreq"],
            buffer_pick_size = config["bufferPickSize"],
            n_epochs = config["nEpochs"],
            patch_size = config["patchSize"],
            batch_size = config["batchSize"],
            tag = tag,
            rotation = config["useRotation"],
            mirroring = config["useMirroring"],
            scaling_factor = config["scalingFactor"],
            use_original = config["useOriginal"],
            threshold=config["classCountThreshold"],
            config = config
            )

        acc = np.loadtxt("./results/{}/data/accuracy.txt".format(tag), skiprows=1)
        avg_acc = np.std(acc[-5:])

        f1 = np.loadtxt("./results/{}/data/F1_score.txt".format(tag), skiprows=1)
        avg_f1 = np.std(f1[-5:])
        
        with open(f"results/result04.txt","a+") as file:
            np.savetxt(file,np.array([threshold, avg_acc, avg_f1]))
        file.close()    

    except RuntimeError:
        torch.cuda.empty_cache()
        continue

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,4)
loss = np.loadtxt("./results/test28-184443/data/training_loss.txt", skiprows=1)
plt.plot(np.arange(len(loss)), loss)

In [ ]:
plt.rcParams["figure.figsize"] = (20,4)
loss = np.loadtxt("./results/test28-185510/data/training_loss.txt", skiprows=1)
plt.plot(np.arange(len(loss)), loss)

Batch size 5, 256x256, lr 0.002

In [ ]:
plt.rcParams["figure.figsize"] = (20,4)
loss = np.loadtxt("./results/test28-191733/data/training_loss.txt", skiprows=1)
plt.plot(np.arange(len(loss)), loss)

Batch size 7, 256x256, lr 0.002

In [ ]:
plt.rcParams["figure.figsize"] = (20,4)
loss = np.loadtxt("./results/test28-193412/data/training_loss.txt", skiprows=1)
plt.plot(np.arange(len(loss)), loss)

Batchsize 10, 128x128, lr 0.003

In [ ]:
plt.rcParams["figure.figsize"] = (20,4)
loss = np.loadtxt("./results/test28-193412/data/training_loss.txt", skiprows=1)
plt.plot(np.arange(len(loss)), loss)

Batchsize 15, 128x128

In [ ]:
plt.rcParams["figure.figsize"] = (20,4)
loss = np.loadtxt("./results/test31-105802/data/training_loss.txt", skiprows=1)
plt.plot(np.arange(len(loss)), loss)

256 x 256, 1 epoch, batch size 2, scaling factor 6, New: Selecting for images with multiple classes (threshold 3)

In [ ]:
plt.rcParams["figure.figsize"] = (20,4)
loss = np.loadtxt("./results/test31-120614/data/training_loss.txt", skiprows=1)
plt.plot(np.arange(len(loss)), loss)

256 x 256, 2 epochs, batch size 5, scaling factor 4, using also original. New: Selecting for images with multiple classes (threshold 3), usage of downscaled dataset only

256 x 256, 1 epoch, batch size 10, scaling factor 4, using also original. New: Selecting for images with multiple classes (threshold 3), usage of downscaled dataset only

In [ ]:
xs = [1,2,3,4,5,7]
accs = [0.471,0.546,0.447,0.514,0.489,0.482]
acc_errs = [0.058,0.125,0.092,0.081,0.068,0.103]
plt.errorbar(xs,accs,acc_errs)
plt.ylim(0.3,0.7)

In [ ]:
'''
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        #inputs = torch.sigmoid(inputs)
        targets = torch.nn.functional.one_hot(targets,24).permute(0,3,1,2).float()       
        
        #flatten label and prediction tensors
        inputs = inputs.flatten()
        targets = targets.flatten()
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice
'''

#crit = CriterionCombo
crit = DiceLoss

model = ut.UNet(3, 24, [64, 128, 256, 512]).to(device)
model.apply(initialize_weights)
optimizer = optim.Adam(model.parameters(), lr = 0.0003)

#Data set and data loader for the training set
DS_training = ImSegDataSet(PathToDataSet="./data/train_set/",mode = "train")
DL_training = DataLoader(dataset=DS_training,batch_size=1,shuffle=True)

#Data set and data loader for the validation set
print("\n")
DS_validation = ImSegDataSet(PathToDataSet="./data/validation_set/",mode = "validation")
DL_validation = DataLoader(dataset=DS_validation,batch_size=5)

'''
result = train(
    model = model, 
    dataloader_training = DL_training, 
    dataLoader_validation = DL_validation,
    optimizer = optimizer, 
    criterion = crit, 
    device = device, 
    buffer_size = 5, 
    buffer_update_freq = 5,
    buffer_pick_size = 3,
    n_epochs = 5,
    patch_size = 32,
    tag = "test_run_1_2023-03-20"
    )
'''

result = train(
    model = model, 
    dataloader_training = DL_training, 
    dataLoader_validation = DL_validation,
    optimizer = optimizer, 
    criterion = crit, 
    device = device, 
    buffer_size = 10, 
    buffer_update_freq = 1,
    buffer_pick_size = 10,
    n_epochs = 1,
    patch_size = 512,
    batch_size = 3,
    tag = "test{}".format(time.strftime("%d-%H%M%S", time.localtime(time.time()))),
    rotation = False,
    mirroring = False,
    scaling_factor = 10,
    use_original = True,
    threshold=3
    )